In [ ]:
import pandas as pd
import json
import requests
import psycopg2

def extract():
    username = "Larry23"
    url = f"https://lichess.org/api/games/user/{username}"

    params = {
        "max" : 50,
        "pgnInJson": True,
        "moves": True
    }

    headers = {
        "Accept": "application/x-ndjson"
    }

    response = requests.get(url, params=params, headers=headers)

    with open("raw_games.ndjson", "w") as f:
        f.write(response.text)

    print("Games downloaded!")

extract()

Games downloaded!


In [6]:
def transform():

    records = []

    with open("raw_games.ndjson") as f:
        for line in f:
            game = json.loads(line)

            if game.get("source") == "friend":

                players = game["players"]
                white = players["white"]
                black = players["black"]

                record = {
                    "game_id": game["id"],
                    "rated": game["rated"],
                    "speed": game["speed"],
                    "game_outcome": game["status"],
                    "time_control": game["clock"]["initial"],
                    "white_player": white["user"]["name"],
                    "black_player": black["user"]["name"],
                    "white_rating": white["rating"],
                    "black_rating": black["rating"],
                    "winner": game["winner"],
                    "moves_count": len(game.get("moves").split())
                }

                records.append(record)

        df = pd.DataFrame(records)

        df.to_csv("games.csv", index=False)
        print("CSV file created!")

transform()        

CSV file created!


In [ ]:
def load():
    
    df = pd.read_csv("games.csv")

    conn = psycopg2.connect(
        dbname = "chessdb",
        user="chessuser",
        password="chesspass",
        host="localhost"
    )

    cur = conn.cursor()

    for _, row in df.iterrows():
        cur.execute("""
                    INSERT INTO games (game_id, rated, speed, game_outcome, time_control, white_player, black_player, white_rating, black_rating, winner, moves_count) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
                    ON CONFLICT (game_id) DO NOTHING;
                    """,(
                            row["game_id"],
                            row["rated"],
                            row["speed"],
                            row["game_outcome"],
                            row["time_control"],
                            row["white_player"],
                            row["black_player"],
                            row["white_rating"],
                            row["black_rating"],
                            row["winner"],
                            row["moves_count"]
                            ))
    conn.commit()
    cur.close()
    conn.close()

    print("Data loaded into PostgreSQL")

